## 5. 토픽 모델링(Topic Modeling)

토픽 모델링(Topic Modeling)이란 기계 학습 및 자연어 처리 분야에서 토픽 모델(Topic Model)이라는 문서 집합의 추상적인 주제를 발견하기 위한 통계적 모델 중 하나로, 텍스트 본문의 숨겨진 의미구조를 발견하기 위해 사용되는 텍스트 마이닝 기법입니다.

### 1. 잠재 의미 분석(Latent Semantic Analysis, LSA)

BoW에 기반한 단어 문서 행렬이나 TF-IDF는 기본적으로 단어의 빈도 수를 이용한 수치화 방법이기 때문에 단어의 의미를 고려하지 못합니다(이를 토픽 모델링 관점에서는 단어의 토픽을 고려하지 못한다고 합니다). 이를 위한 대안으로 단어 문서 행렬의 잠재된(Latent) 의미를 이끌어내는 방법으로 잠재 의미 분석(Latent Semantic Analysis)이라는 방법이 있습니다. 이 방법은 선형대수학의 특이값 분해(Singular Value Decomposition)를 통해 행할 수 있습니다.

#### 1. Reduced SVD

SVD에서 일부 벡터들을 삭제하는 것을 데이터 차원을 줄인다고도 말하는데, 데이터의 차원을 줄이게 되면 계산 비용이 낮아지는 효과를 얻을 수 있습니다. 또, 상대적으로 중요하지 않은 정보를 삭제하는 효과를 갖고 있는데, 이는 영상 처리 분야에서는 노이즈를 제거한다는 의미를 갖고, 자연어 처리 분야에서는 설명력이 낮은 정보를 삭제한다는 의미를 갖고 있습니다.

#### 2. 잠재 의미 분석(Latent Semantic Analysis, LSA)

기존의 단어 문서 행렬이나 단어 문서 행렬에 단어의 중요도에 따른 가중치를 주었던 TF-IDF 행렬은 단어의 의미를 전혀 고려하지 못했습니다. LSA는 기본적으로 단어 문서 행렬이나 TF-IDF 행렬에 SVD를 사용하여 차원을 축소시키고, 단어들의 잠재적인 의미를 끌어낸다는 아이디어를 갖고 있습니다.

#### 3. 실습을 통한 이해

사이킷 런에서는 Twenty Newsgroups라는 20개의 다른 주제를 가진 뉴스 데이터를 제공합니다. 이제 해당 데이터를 통해서 직접 LSA를 통해 토픽 모델링을 수행해보도록 하겠습니다.

##### 1. 뉴스데이터에 대한 이해

In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers','quotes'))

documents = dataset.data
len(documents)

11314

해당 데이터는 총 11,314개의 데이터를 갖고 있습니다. 이 중 한 데이터를 출력해보겠습니다.

In [2]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [3]:
print(documents[1])








Yeah, do you expect people to read the FAQ, etc. and actually accept hard
atheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out
of steam!







Jim,

Sorry I can't pity you, Jim.  And I'm sorry that you have these feelings of
denial about the faith you need to get by.  Oh well, just pretend that it will
all end happily ever after anyway.  Maybe if you start a new newsgroup,
alt.atheist.hard, you won't be bummin' so much?






Bye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) 
--
Bake Timmons, III


보시다 시피 많은 특수문자가 있는 영어문장으로 구성되어져 있습니다. 사이킷 런이 제공하는 뉴스 데이터에서 target_name에 본래 이 뉴스 데이터가 어떤 20개의 카테고리를 갖고 있었는지가 저장되어 있습니다.

In [4]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


##### 2. 텍스트 전처리

1. 알파벳을 제외한 구두점, 숫자, 특수 문자를 공백으로 제거하기
2. 길이가 짧은 단어는 제거하기
3. 알파벳을 소문자로 바꾸기

In [5]:
news_df = pd.DataFrame(documents, columns=['document'])

news_df['clean_doc'] = news_df['document'].str.replace('[^a-zA-Z#]', ' ')
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

news_df.head(2)

,document,clean_doc
0,Well i'm not sure about the story nad it did s...,well sure about story seem biased what disagre...
1,"\n\n\n\n\n\n\nYeah, do you expect people to re...",yeah expect people read actually accept hard a...


이제 토큰화와 불영어 제거를 하겠습니다.

In [6]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [7]:
tokenized_doc.head()

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

##### 3. TF-IDF 행렬 만들기

불용어 제거를 위해 토큰화 작업을 수행했지만, TfidfVectorizer는 기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용합니다. 그렇기 때문에 다시 토큰화를 역으로 하는 작업을 수행해보도록 하겠습니다. 

In [8]:
detokenized_doc = []

for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
    
news_df['clean_doc'] = detokenized_doc

In [9]:
news_df['clean_doc'].head()

0    well sure story seem biased disagree statement...
1    yeah expect people read actually accept hard a...
2    although realize principle strongest points wo...
3    notwithstanding legitimate fuss proposal much ...
4    well change scoring playoff pool unfortunately...
Name: clean_doc, dtype: object

이제 TF-IDF 행렬을 만들텐데 계산 시간 문제로 단어는 1000개만 가지고 사용하겠습니다.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_features=1000,
                               max_df=0.5, smooth_idf=True)
## max_df 단어장에 포함되기 위한 최대 빈도
## 피처를 만들 때 0으로 나오는 항목에 대해 작은 값을 
## 더해서(스무딩을 해서) 피처를 만들지 아니면 그냥 생성할지를 결정

X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape

(11314, 1000)

smooth_idf = True
$$idf = log(\frac{N+1}{N_w + 1} + 1)$$

smooth_idf = False
$$idf = log(\frac{N}{N_w}+1)$$

##### 4. 토픽 모델링(Topic Modeling)

이제 TF-IDF 행렬을 다수의 행렬로 분해해보도록 하겠습니다. 사이킷런의 TruncatedSVD를 사용합니다. 원래 기존 뉴스 데이터 자체가 20개의 다른 뉴스 카테고리를 갖고 있었기 때문에, 텍스트 데이터에 20개의 토픽 모델링을 시도해보겠습니다.

In [11]:
from sklearn.decomposition import TruncatedSVD

svd_model  = TruncatedSVD(n_components=20, 
                          algorithm='randomized', n_iter=100,
                         random_state=122)

svd_model.fit(X)
len(svd_model.components_)

20

In [13]:
svd_model.transform(X).shape

(11314, 20)

In [16]:
svd_model.components_.shape

(20, 1000)

In [20]:
vectorizer.get_feature_names()[:10], len(vectorizer.get_feature_names())

(['ability',
  'able',
  'accept',
  'access',
  'according',
  'account',
  'action',
  'actions',
  'actual',
  'actually'],
 1000)

이제 분류된 20개의 토픽 각각에서 가장 중요한 단어 5개씩 출력해보겠습니다.

In [ ]:
a= [(1,3), ()]

In [45]:
terms = vectorizer.get_feature_names()
## 단어들

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key=lambda x:x[1], reverse=True)[:5]
    print("Topic "+str(i+1)+": ", end='')
    for t in sorted_terms:
        print(t[0], end=' ')
    print(' ')

Topic 1: like know people think good  
Topic 2: thanks windows card drive mail  
Topic 3: game team year games season  
Topic 4: drive scsi hard disk card  
Topic 5: windows file window files program  
Topic 6: chip government mail space information  
Topic 7: like bike know chip sounds  
Topic 8: card sale video monitor offer  
Topic 9: know card chip video government  
Topic 10: good know time bike jesus  
Topic 11: think chip good thanks clipper  
Topic 12: thanks right problem good bike  
Topic 13: good people windows know file  
Topic 14: space think know nasa problem  
Topic 15: space good card people time  
Topic 16: people problem window time game  
Topic 17: time bike right windows file  
Topic 18: time problem file think israel  
Topic 19: file need card files problem  
Topic 20: problem file thanks used space  


#### 5. LSA의 장단점

정리해보면 LSA는 쉽고 빠르게 구현이 가능할 뿐만 아니라 __단어의 잠재적인 의미__ 를 끌어낼 수 있어 토픽 모델링, 문서의 유사도 계산 등에서 좋은 성능을 보여준다는 장점을 갖고 있습니다. 하지만 SVD의 특성상 이미 계산된 LSA에 새로운 데이터를 추가하여 계산하려고 하면 보통 처음부터 다시 계산해야 합니다. 이는 최근 LSA 대신 word2vec 등 단어의 의미를 벡터화할 수 있는 다른 방법론인 뉴럴 네트워크 기반의 방법론이 각광받는 이유입니다.
